In [1]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown
from black_scholes import black_scholes_price, compute_greeks

def simulate_delta_hedging_gbm(S0, K, T, r, sigma_imp, sigma_mkt, option_type, steps, n_sim):
    dt = T / steps
    pnl_list = []

    for sim in range(n_sim):
        S_path = [S0]
        for _ in range(steps):
            z = np.random.normal()
            S_next = S_path[-1] * np.exp((r - 0.5 * sigma_mkt**2) * dt + sigma_mkt * np.sqrt(dt) * z)
            S_path.append(S_next)
        S_path = np.array(S_path)
        times = np.linspace(0, T, steps + 1)

        cash_account = 0
        delta_prev = 0

        for i, (t, S) in enumerate(zip(times[:-1], S_path[:-1])):
            T_remaining = T - t
            delta, _, _ = compute_greeks(S, K, T_remaining, r, sigma_imp, option_type)

            d_delta = delta - delta_prev
            cash_account -= d_delta * S
            cash_account *= np.exp(r * dt)
            delta_prev = delta

        portfolio_value = delta_prev * S_path[-1] + cash_account
        payoff = max(S_path[-1] - K, 0) if option_type == "call" else max(K - S_path[-1], 0)
        pnl = portfolio_value - payoff
        pnl_list.append(pnl)

    pnl_array = np.array(pnl_list)
    return pnl_array

def plot_pnl_distribution(pnl_array):
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=pnl_array, nbinsx=50, marker_color='skyblue'))
    fig.update_layout(
        title="Distribution du PnL de la stratégie Delta Hedging",
        xaxis_title="PnL (€)",
        yaxis_title="Fréquence",
        bargap=0.1,
        template="plotly_white"
    )
    fig.show()

@interact(
    S0=FloatSlider(min=50, max=150, step=1, value=100, description='Spot initial'),
    K=FloatSlider(min=50, max=150, step=1, value=100, description='Strike'),
    T=FloatSlider(min=0.01, max=1, step=0.01, value=30/365, description='Maturité (ans)'),
    r=FloatSlider(min=0, max=0.1, step=0.001, value=0.01, description='Taux sans risque'),
    sigma_imp=FloatSlider(min=0.05, max=1, step=0.01, value=0.2, description='Vol Implicite'),
    sigma_mkt=FloatSlider(min=0.05, max=1, step=0.01, value=0.25, description='Vol Marché'),
    option_type=Dropdown(options=['call', 'put'], value='call', description='Option type'),
    steps=IntSlider(min=10, max=100, step=5, value=30, description='Steps (jours)'),
    n_sim=IntSlider(min=100, max=5000, step=100, value=1000, description='Simulations')
)
def run_simulation(S0, K, T, r, sigma_imp, sigma_mkt, option_type, steps, n_sim):
    pnl = simulate_delta_hedging_gbm(S0, K, T, r, sigma_imp, sigma_mkt, option_type, steps, n_sim)
    print(f"PnL moyen : {np.mean(pnl):.4f} €")
    print(f"PnL std : {np.std(pnl):.4f} €")
    plot_pnl_distribution(pnl)
    

interactive(children=(FloatSlider(value=100.0, description='Spot initial', max=150.0, min=50.0, step=1.0), Flo…